## Relative Size Factor

#### Getting the data ready

In [1]:
import pandas as pd
pc_df = pd.read_excel("PCard_Data2009-2014.xlsx")
pc_df['Amount'] = pc_df['Amount'].abs()
pc_df['Description'] = pc_df['Description'].str.replace(', Not Elsewhere Classified','')
pc_df['Description'] = pc_df['Description'].str.replace('Not Elsewhere Classified','')
pc_df.head()

,Date,Amount,Merchant,State,Description,Agency
0,2009-01-02,1700.00,WASHINGTON HISPANIC,MD,Advertising Services,Department of Motor Vehicles
1,2009-01-02,23.16,STAPLES DIRECT00209908,CA,"Stationery,Office Supplies,Printing/Writing Paper",Metropolitan Police Department
2,2009-01-02,9.48,AMAZON.COM,WA,Book Stores,Department of Property Management
3,2009-01-02,24.61,FEDEX KINKO'S #0671,DC,"Quick-Copy, Reproduction and Blueprinting Serv...",Fire & Emergency Medical Services
4,2009-01-02,31.65,J&R SOUND/MAILORDER,NY,Direct Marketing - Catalog Merchants,Office of the State Superintendent of Education


#### Simple Demonstration of the idea

Create a list of numbers <br>
Using sort(), the numbers are sorted in ascending order. <br>
Using indexing, get the biggest number and the next big number. <br>
Divide the biggest number by the next big number <br>

In [2]:

numbers = [8000,200,200,150]
numbers.sort()
Largest = numbers[-1]
Semi_Largest = numbers[-2]
RSF = Largest/Semi_Largest
print(RSF)

40.0


Let's find the RSF for one vendor.

In [3]:
# Filter the existing dataframe to retrieve rows that has "Washington Hispanic" for the Merchant column
pc_df.loc[pc_df['Merchant'] == 'WASHINGTON HISPANIC']

,Date,Amount,Merchant,State,Description,Agency
0,2009-01-02,1700.0,WASHINGTON HISPANIC,MD,Advertising Services,Department of Motor Vehicles
27407,2009-12-16,2700.0,WASHINGTON HISPANIC,MD,Advertising Services,Office of Human Rights
40860,2010-05-28,350.0,WASHINGTON HISPANIC,MD,Advertising Services,Office of the Tenant Advocate
64337,2011-02-08,750.0,WASHINGTON HISPANIC,MD,News Dealers & Newsstands,Commission on Arts & Humanities
71560,2011-04-26,700.0,WASHINGTON HISPANIC,MD,News Dealers & Newsstands,Commission on Arts & Humanities
81239,2011-08-11,300.0,WASHINGTON HISPANIC,MD,News Dealers & Newsstands,DC Public Library
98559,2012-02-08,700.0,WASHINGTON HISPANIC,MD,News Dealers & Newsstands,Office of the Secretary
127387,2012-12-19,250.0,WASHINGTON HISPANIC,MD,News Dealers & Newsstands,Department on Disability Services
143759,2013-06-25,800.0,WASHINGTON HISPANIC,MD,News Dealers & Newsstands,District Department of Transportation
153093,2013-09-30,120.0,WASHINGTON HISPANIC,MD,News Dealers & Newsstands,OFFICE OF PEOPLE'S COUNSEL


In [4]:
# Create a dataframe that contains only the rows filtered in the above step.
single_RSF_df = pc_df.loc[pc_df['Merchant'] == 'WASHINGTON HISPANIC']

DataFrame.nlargest returns the first rows ordered by columns in descending order, <br> meaning the biggest numbers at the top.

In [5]:
# This returns the two highest numbers
single_RSF_df['Amount'].nlargest(2)

27407     2700.0
177215    1800.0
Name: Amount, dtype: float64

In [6]:
RSF = single_RSF_df['Amount'].nlargest(2).values[0]/single_RSF_df['Amount'].nlargest(2).values[1]
RSF

1.5

Let's check the work.

In [7]:
single_RSF_df['Amount']

0         1700.0
27407     2700.0
40860      350.0
64337      750.0
71560      700.0
81239      300.0
98559      700.0
127387     250.0
143759     800.0
153093     120.0
167515     700.0
173511     277.0
177215    1800.0
184550     336.0
Name: Amount, dtype: float64

In [8]:
2700/1800

1.5

Complications that weren't accounted for this time. <br>
* Delete amounts equal to or lower than zero  <br>( I actually took the absolute value of the amounts this time around.) <br>
* Delete all the subsets with only one record
* Account for subsets where the largest and the second largest numbers are equal.

In addition to the Relative Size Factor, <br>
one could also try writing functions for the following <br>
* the largest amount divided by the average amount <br>
* the largest amount divided by the average amount where the average excludes the largest number
* the smallest number divided by the average (used when trying to identify understatements)

In [1]:
import pandas as pd
from datetime import datetime
initial_data = pd.read_excel("PCard_Data2009-2014.xlsx")
initial_data.info()


#Option A - Ihaveamodel3's way; Groupby

optionA_data = pd.DataFrame(initial_data)

def agg_func_A(vals):
    vals2 = vals.nlargest(2).values
    if len(vals2)>1:
        return vals2[0]/vals2[1]
    else:
        return 0

start = datetime.now()
RSF = optionA_data.groupby("Merchant", as_index=False)["Amount"].agg(agg_func_A)
optionA_data = pd.merge(optionA_data,RSF, on="Merchant", how="left",suffixes=('_a','_b'))
optionA_data = optionA_data.rename({"Amount_a":"Amount","Amount_b":"RSF"}, axis=1)
print(datetime.now()-start)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193791 entries, 0 to 193790
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Date         193791 non-null  datetime64[ns]
 1   Amount       193791 non-null  float64       
 2   Merchant     193752 non-null  object        
 3   State        193562 non-null  object        
 4   Description  193773 non-null  object        
 5   Agency       193791 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 8.9+ MB
0:00:07.853819


In [2]:
optionA_data

,Date,Amount,Merchant,State,Description,Agency,RSF
0,2009-01-02,-1700.00,WASHINGTON HISPANIC,MD,Advertising Services,Department of Motor Vehicles,1.500000
1,2009-01-02,-23.16,STAPLES DIRECT00209908,CA,"Stationery,Office Supplies,Printing/Writing Paper",Metropolitan Police Department,1.142929
2,2009-01-02,9.48,AMAZON.COM,WA,Book Stores,Department of Property Management,1.000000
3,2009-01-02,24.61,FEDEX KINKO'S #0671,DC,"Quick-Copy, Reproduction and Blueprinting Serv...",Fire & Emergency Medical Services,1.252824
4,2009-01-02,31.65,J&R SOUND/MAILORDER,NY,Direct Marketing - Catalog Merchants,Office of the State Superintendent of Education,1.320738
...,...,...,...,...,...,...,...
193786,2014-12-31,2269.01,AMERICAN BUSINESS SUPP,DC,"Office,Photographic,Photocopy,and Microfilm Equip",DC Public Library,1.035224
193787,2014-12-31,2519.78,STANDARD OFFICE SUPPLY,DC,"Stationery, Office & School Supply Stores",Fire & Emergency Medical Services,1.000000
193788,2014-12-31,2894.15,AOP BUSINESS SERVICES,DC,"Stationery,Office Supplies,Printing/Writing Paper",Department of Human Services,1.006686
193789,2014-12-31,4900.00,"IN *CRWP EAGLESYSTEMS,",MD,"Genl Contractors-Residential, and Commercial",DC Public Library,1.116427


In [ ]:
#Option B - learnhtk; apply: very inefficient.
optionB_data= pd.DataFrame(initial_data)

def apply_func_B(row):
    merchant = row["Merchant"]
    data_for_merchant = optionB_data[optionB_data["Merchant"]==merchant]
    vals = data_for_merchant["Amount"].nlargest(2).values
    if len(vals)>1:
        return vals[0]/vals[1]
    else:
        return 0

start = datetime.now()
optionB_data["RSF"] = optionB_data.apply(apply_func_B, axis=1)
print(datetime.now()-start)